# Test NSEG dataset

In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./dataset_eeg_des.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [3]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      2.135436  2.887686  3.465504  3.810436  3.955764   
                2      2.549241  2.254038  3.489210  3.876865  3.767604   
                3      2.436902  3.456774  4.163457  4.401829  4.272075   
                4      1.841611  2.513253  3.124816  3.421180  3.492676   
                5      2.320207  2.961560  3.857012  4.111162  3.868319   
...                         ...       ...       ...       ...       ...   
9       3       20     1.398441  2.604782  3.687751  3.648057  3.009953   
                21     1.571485  1.894192  2.786308  3.305627  3.571761   
                22     1.270886  2.200624  2.915375  3.471395  3.427353   
                23     2.621798  3.373337  3.905828  3.839850  3.868095   
                24     1.494128  3.076038  2.783606  3.572978  3.547337   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      4.053275  3.646478  3.300221  3.606062  3.223825  ...   
                2      3.407996  3.706311  3.790671  3.835916  3.726078  ...   
                3      3.567253  3.214838  3.632954  3.966298  3.774067  ...   
                4      3.306529  3.099885  3.100102  3.221046  3.320871  ...   
                5      3.717754  3.698447  3.367067  3.021535  3.351307  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     3.942856  3.935395  2.950334  3.604765  3.776026  ...   
                21     3.696536  3.393062  2.770488  2.746998  2.790538  ...   
                22     2.811443  3.023582  2.642579  3.219103  3.161054  ...   
                23     3.649109  3.723545  3.259310  3.605300  3.623675  ...   
                24     2.896757  3.371674  3.948618  4.159415  4.232633  ...   

                            492       493       494       495       496  \
id_user session video                                                     
10      1       1     -1.011859 -1.020659 -0.602725 -0.584323 -0.602491   
                2     -0.654046 -0.287855 -0.064083 -0.382718 -0.854156   
                3     -0.102593 -0.080281 -0.138853  0.001005 -0.083322   
                4     -0.990079 -0.927056 -0.745584 -0.978420 -1.031638   
                5     -0.247413 -0.758625 -1.643860 -1.252678 -0.605932   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.389243 -0.347923 -0.460666 -0.483391 -0.797307   
                21    -0.804578 -1.045797 -1.104529 -0.949304 -0.719607   
                22    -0.531922 -0.662631 -0.867511 -1.125926 -0.903615   
                23    -0.544234 -0.343033 -0.551949 -0.976702 -1.112032   
                24    -0.602491  0.003543  0.088560 -0.193022 -0.742112   

                            497       498       499       500  emotion  
id_user session video                                                   
10      1       1     -0.579400 -1.200410 -0.910379 -1.369580        1  
                2     -0.538963 -0.978321 -0.520367 -1.108300        2  
                3     -0.067288 -0.063967  0.042125 -0.123110        3  
                4     -0.682465 -0.665910 -1.033460 -1.373817        0  
                5     -0.750933 -1.202377 -0.686867 -1.160245        2  
...                         ...       ...       ...       ...      ...  
9       3       20    -0.593601 -0.401147 -0.498099 -0.751132        0  
                21    -0.668664 -0.650504 -0.838495 -1.361561        2  
                22    -1.054831 -0.916811 -1.142544 -1.251611        0  
                23    -0.834240 -0.739685 -0.623760 -1.128035        1  
                24    -1.026718 -0.977749 -0.960097 -1.363363        0  

[1080 rows x 502 columns]

In [4]:
def preprocessing_for_LSTM(X_train, X_test, y_train, y_test):
    X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
    y_train = to_categorical(y_train)
    X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
    y_test = to_categorical(y_test)
    return X_train, X_test, y_train, y_test

# Subject dependent

In [5]:
def subject_dependent_split(df,session):
    df = df.reset_index()
    np.random.seed(75)
    test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
    df_sess = df.loc[df['session'] == session]
    X_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    X_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    return X_train, X_test, y_train, y_test

In [7]:
accuracy_test = []
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    X_trainSD, X_testSD, y_trainSD, y_testSD = preprocessing_for_LSTM(X_trainSD, X_testSD, y_trainSD, y_testSD)
    model = Sequential()
    model.add(LSTM(units=64, input_shape=(X_trainSD.shape[1], 1)))
    model.add(Dense(units=4, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_trainSD, y_trainSD, epochs=10, batch_size=32)
    loss, accuracy = model.evaluate(X_testSD, y_testSD)
    accuracy_test.append(accuracy)

Epoch 1/10
8/8 [==============================] - 5s 288ms/step - loss: nan - accuracy: 0.2458
Epoch 2/10
8/8 [==============================] - 2s 276ms/step - loss: nan - accuracy: 0.2500
Epoch 3/10
8/8 [==============================] - 2s 289ms/step - loss: nan - accuracy: 0.2500
Epoch 4/10
8/8 [==============================] - 2s 283ms/step - loss: nan - accuracy: 0.2500
Epoch 5/10
8/8 [==============================] - 2s 272ms/step - loss: nan - accuracy: 0.2500
Epoch 6/10
8/8 [==============================] - 2s 308ms/step - loss: nan - accuracy: 0.2500
Epoch 7/10
8/8 [==============================] - 2s 268ms/step - loss: nan - accuracy: 0.2500
Epoch 8/10
8/8 [==============================] - 2s 279ms/step - loss: nan - accuracy: 0.2500
Epoch 9/10
8/8 [==============================] - 2s 276ms/step - loss: nan - accuracy: 0.2500
Epoch 10/10
4/4 [==============================] - 1s 102ms/step - loss: nan - accuracy: 0.2500
Epoch 1/10
8/8 [==============================] -

In [9]:
print("ACCURACY: ", np.array(accuracy_test).mean())

ACCURACY:  0.25555556019147235
LOSS:  nan


# Subject-Biased Experiment


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=22)
X_train, X_test, y_train, y_test = preprocessing_for_LSTM(X_train, X_test, y_train, y_test)

In [11]:
del model
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=4, activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
27/27 [==============================] - 8s 221ms/step - loss: nan - accuracy: 0.2431
Epoch 2/10
27/27 [==============================] - 6s 217ms/step - loss: nan - accuracy: 0.2442
Epoch 3/10
27/27 [==============================] - 6s 218ms/step - loss: nan - accuracy: 0.2442
Epoch 4/10
27/27 [==============================] - 6s 211ms/step - loss: nan - accuracy: 0.2442
Epoch 5/10
27/27 [==============================] - 6s 221ms/step - loss: nan - accuracy: 0.2442
Epoch 6/10
27/27 [==============================] - 6s 227ms/step - loss: nan - accuracy: 0.2442
Epoch 7/10
27/27 [==============================] - 6s 240ms/step - loss: nan - accuracy: 0.2442
Epoch 8/10
27/27 [==============================] - 6s 224ms/step - loss: nan - accuracy: 0.2442
Epoch 9/10
27/27 [==============================] - 6s 222ms/step - loss: nan - accuracy: 0.2442
Epoch 10/10
27/27 [==============================] - 6s 227ms/step - loss: nan - accuracy: 0.2442


In [12]:
loss, accuracy = model.evaluate(X_test, y_test)

7/7 [==============================] - 1s 96ms/step - loss: nan - accuracy: 0.2731


# Subject Indipendent

In [ ]:
def subject_independent_split(df,session):
    df = df.reset_index()
    df_sess = df.loc[df['session'] == session]
    groups = df_sess['id_user']
    X = df_sess.set_index(['id_user','session','video']).drop('emotion',axis=1)
    y = df_sess.set_index(['id_user','session','video']).emotion
    return X, y, groups

# TO DO: da implementare kfold
https://stackoverflow.com/questions/48085182/cross-validation-in-keras


"""
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow.keras.models import * 
from tensorflow.keras.layers import * 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

def buildmodel():
    model= Sequential([
        Dense(10, activation="relu"),
        Dense(5, activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return(model)

estimator= KerasRegressor(build_fn=buildmodel, epochs=100, batch_size=10, verbose=0)
kfold= RepeatedKFold(n_splits=5, n_repeats=100)
results= cross_val_score(estimator, x, y, cv=kfold, n_jobs=2)  # 2 cpus
results.mean()  # Mean MSE
"""

In [ ]:
scores_test = []
logo = LeaveOneGroupOut()
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    X_SI = np.array(X_SI).reshape((X_SI.shape[0], X_SI.shape[1], 1))
    y_SI = to_categorical(y_SI)
    del model
    model = Sequential()
    model.add(LSTM(units=64, input_shape=(int(X_SI.shape[1])-15, 1)))
    model.add(Dense(units=4, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    scores_gbc = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())